In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
np.random.seed(1)

In [ ]:
mnist=fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
# print(mnist.DESCR)
# print(mnist)

In [ ]:
# initializing data
x,y=mnist.data,mnist.target
# print(y)
x_train_orig,x_test_orig=np.array(x[:60000]),np.array(x[60000:])
y_train_orig,y_test_orig=np.array(y[:60000]),np.array(y[60000:])

In [ ]:

# testing data
  # plt.imshow(x_train_orig[0].reshape(28,28))
  # print(y_train_orig[0])
  # print(x_train_orig.shape)
# number of images
m=x_train_orig.shape[0]
#flattening data
x_train_flatten=x_train_orig.reshape(x_train_orig.shape[0],-1)
x_test_flatten=x_test_orig.reshape(x_test_orig.shape[0],-1)
x_train=x_train_flatten/255
x_test=x_test_flatten/255
y_train=y_train_orig.reshape(y_train_orig.shape[0],-1)
y_test=y_test_orig.reshape(y_test_orig.shape[0],-1)
x_train=x_train.T
y_train=y_train.T
y_train=y_train.astype(float)
y_test=y_test.astype(float)
y_temp=np.zeros((10,y.shape[0]))
# print(y[0])
for i in range(100):
  for j in range(10):
    if(j==int(y[i])):
      # print(str(y[i])+str(j))
      y_temp[j,i]=1
# print(y_temp.shape)
print(y_temp[4][0])
y_train,y_test=y_temp[:,:60000],y_temp[:,60000:]
print(y_train.shape)
dim_layers=[]
dim_layers.append(x_train.shape[0])
print('!last layer must have 10 nodes!')
layers_len=int(input("No.of hidden layers: "))
for i in range (layers_len):
  n=input("No. of nodes in layer "+str(i+1)+": ")
  dim_layers.append(int(n))
layers_len+=1
print(dim_layers)

In [ ]:
def sigmoid(Z):
  A=1/(1+np.exp(-Z))
  return A
def relu(Z):
  return np.where(Z>0,Z,0.01*Z)
def softmax(Z):
  # print('hello')
  A=np.exp(Z)/np.sum(np.exp(Z),axis=0,keepdims=True)
  return A
def deriv_relu(Z):
  # print(type(Z),'hi')
  return np.where(Z>0,1,0.01)
def deriv_softmax(Z):
  # print('hi')
  length=10
  dZ=np.zeros((60000,10))
  Z=np.transpose(Z)
  for row in range (0,60000):
          den=(np.sum(np.exp(Z[row,:])))*(np.sum(np.exp(Z[row,:])))
          for col in range (0,10):
              sums=0
              for j in range (0,10):
                  if (j!=col):
                      sums=sums+(np.math.exp(Z[row,j]))

              dZ[row,col]=(np.math.exp(Z[row,col])*sums)/den
  dZ=np.transpose(dZ)
  Z=np.transpose(Z)

    # assert (dZ.shape == Z.shape)
  return dZ

In [ ]:
def initialize_parameters(dim_layers):
    # hidden_layers contains dimensions of each hidden layer
  parameters={}
  for l in range(1,layers_len):
    parameters['W'+str(l)]=np.random.randn(dim_layers[l],dim_layers[l-1])*0.001
    parameters['b'+str(l)]=np.zeros((dim_layers[l],1))
  return parameters

In [ ]:
def linear_activation_forward(A_prev,W,b,activation):
  # print(str(W.shape)+"w's shape")
  # print(str(A_prev.shape)+"A's shape")
  Z=np.dot(W,A_prev)+b
  if activation=='sigmoid':
    A=sigmoid(Z)
  elif activation=='relu':
    A=relu(Z)
  elif activation=='softmax':
    A=softmax(Z)
  cache=[]
  cache=A_prev,Z
  return A,cache

In [ ]:
# def one_hot(y_train):
#     onehot_y = np.zeros((y_train.size, int(y_train.max()) + 1))
#     onehot_y[np.arange(y_train.size), y_train] = 1
#     onehot_y = onehot_y.T#
# return onehot_y

In [ ]:
def forward_model(dim_layers,parameters,layers_len,activation):
  caches=[]
  A_prev=x_train

  for l in range(1,layers_len-1):
    W,b=parameters['W'+str(l)],parameters['b'+str(l)]
    A_prev,cache=linear_activation_forward(A_prev,W,b,'relu')
    caches.append(cache)

  W,b=parameters['W'+str(layers_len-1)],parameters['b'+str(layers_len-1)]
  AL,cache=linear_activation_forward(A_prev,W,b,'softmax')
  caches.append(cache)

  # AL=np.array(AL[:,:1])
  # ycap=AL.argmax(axis=0)#returns index of max element
  # print(ycap)
  return AL,caches,parameters
# AL,caches,parameters=forward_model(dim_layers,layers_len,'relu')

def backward_model(AL,parameters,cache,activation):
  dAL=-(np.divide(y_train,AL)-np.divide(1-y_train,AL))
  # A,Z=zip(*caches)
  dZ,db,dW,dA=[],[],[],[]
  grads={}
  dA=dAL #dA4

  for l in reversed(range(1,layers_len)):#4,3,2(L=5)
    A=cache[l-1][0] #A4 3 2
    Z=cache[l-1][1] #Z4 3 2
    W,b=parameters['W'+str(l)],parameters['b'+str(l)] # W4,b4
    # print(W.shape)
    if(l==layers_len-1):
      dZ=dA*deriv_softmax(Z)
    else:
      dZ=dA*deriv_relu(Z) #Z3,Z2
    dA=np.dot(W.T,dZ) #dA of prev layer ie layer 2 (1 based indexing)
    db=(1/m)*np.sum(dZ,axis=0,keepdims=True)
    dW=(1/m)*np.dot(dZ,A.T) #A2,A1
    # print(str(dW.shape)+'bckwrd')
    grads['dW'+str(l)]=dW
    grads['db'+str(l)]=db

  return dA,dZ,grads

def cross_entropy(AL,y_train):
  return - np.sum(np.log(AL)*(y_train),axis=1)

def final_cost(AL,y_train):
  return np.mean(cross_entropy(AL,y_train))

def plot_costs(costs, learning_rate):
  plt.plot(np.squeeze(costs))
  plt.ylabel('cost')
  plt.xlabel('iterations (per hundreds)')
  plt.title("Learning rate =" + str(learning_rate))
  plt.show()

In [ ]:
def update_params(parameters,grads,learning_rate):
  for l in range(1,layers_len): #123
    dW=grads['dW'+str(l)]
    db=grads['db'+str(l)]
    # print(dW.shape)
    parameters['W'+str(l)]=parameters['W'+str(l)]-(learning_rate*dW)
    parameters['b'+str(l)]=parameters['b'+str(l)]-(learning_rate*db)
    # print(dW,db)
  return parameters
# parameters=update_params(parameters,grads,learning_rate)

In [ ]:
#prolly final block
# num_iterations=int(input("Enter number of iterations"))
# learning_rate=float(input("Enter magnitude of learning rate"))
num_iterations=300
learning_rate=0.01
costs=[]
parameters={}
parameters=initialize_parameters(dim_layers)
for i in range (0,num_iterations):
  AL,caches,parameters=forward_model(dim_layers,parameters,layers_len,'relu')
  dA,dZ,grads=backward_model(AL,parameters,caches,'relu')
  parameters=update_params(parameters,grads,learning_rate)
  cost=final_cost(AL,y_train)
  if i % 10 == 0 or i == num_iterations - 1:
    print("Cost after iteration {}: {}".format(i, cost))
  if i % 10 == 0 or i == num_iterations:
    costs.append(cost)
plot_costs(costs, learning_rate=0.01)
print(AL[5][0])
print(AL.argmax(axis=0))